In [1]:
pip install -U pandasql

Requirement already up-to-date: pandasql in c:\users\crie072\anaconda3\lib\site-packages (0.7.3)
Note: you may need to restart the kernel to use updated packages.


In [260]:
import pandas as pd
import numpy as np
from pathlib import Path
import csv
import xlrd
import io
from zipfile import ZipFile
from datetime import date, timedelta, time, datetime

from pandasql import sqldf

from pandas.tseries.offsets import MonthEnd

In [261]:
today_yyyymmdd = date.today().strftime("%Y%m%d")
print(today_yyyymmdd)

20200818


In [262]:
#pandasの表示列数を増やす
import pandas as pd
pd.set_option('display.max_columns', 100)

In [263]:
data_dir = Path(r"C:\Users\crie072\Documents\01_データ関連\Adhoc\202008_Web明細")

#input
input_new = "WEBMEISAI_CUST"
input_all = "WEBMEISAI_CUST_ALL"

input_cust = "TOTAL_CUST"
input_cust_all = "TOTAL_CUST_ALL"

#out_put
output = "WEBMEISAI_CUST_ALL"
output2 = "TOTAL_CUST_ALL"


#output(result)
result = "Web明細Overview"
result_cha = "Web明細新規登録数_by_channel"
result_card = "Web明細新規登録数_by_card"
result_age = "Web明細新規登録数_by_age"

①Web明細登録データの読込＆集計

In [264]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
df = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
#文字化けするので修正
#df1['age'] = df1['age'].str.replace('?','-')

#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')#文字化けするので修正
#df2['age'] = df2['age'].str.replace('～','-')

#df = pd.concat([df1,df2])
#上書き更新するので注意
#df.to_csv(data_dir /f'{output}.csv', header=True ,index=None, encoding='shift-jis')

In [265]:
df = df[df['cp']>= 1803] #2018年3月以降に限定
df

,cp,year,month,card,channel,age,gender,new_account,no_of_account
0,1803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8
1,1803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1
2,1803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15
3,1803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2
4,1803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17
...,...,...,...,...,...,...,...,...,...
8747,2005,2020,5,外商,郵送,75-79歳,男性,1,43
8748,2005,2020,5,外商,郵送,80歳以上,女性,0,59
8749,2005,2020,5,外商,郵送,80歳以上,女性,1,11
8750,2005,2020,5,外商,郵送,80歳以上,男性,0,92


In [266]:
print(df.shape)
print(df.index)
print(df.columns)
print(df.dtypes)

(8752, 9)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            8742, 8743, 8744, 8745, 8746, 8747, 8748, 8749, 8750, 8751],
           dtype='int64', length=8752)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'new_account', 'no_of_account'],
      dtype='object')
cp                int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
new_account       int64
no_of_account     int64
dtype: object


In [267]:
#日付データの変換
df['cp'] = df['cp'] + 200000
df['cp_date'] = pd.to_datetime(df['cp'], format="%Y%m") + MonthEnd(1)
df.head()

,cp,year,month,card,channel,age,gender,new_account,no_of_account,cp_date
0,201803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8,2018-03-31
1,201803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1,2018-03-31
2,201803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15,2018-03-31
3,201803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2,2018-03-31
4,201803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17,2018-03-31


#日付データの変換
df['cp_date'] = (df['cp'] + 200000)*100 + 1
df['cp_date']=pd.to_datetime(df['cp_date'], format='%Y%m%d')
df.head()

In [268]:
#日付から時間を削除
df['cp_date'] = df['cp_date'].dt.strftime('%Y/%m/%d')
df.head()

,cp,year,month,card,channel,age,gender,new_account,no_of_account,cp_date
0,201803,2018,3,ゴールド,オンライン,25-29歳,女性,0,8,2018/03/31
1,201803,2018,3,ゴールド,オンライン,25-29歳,女性,1,1,2018/03/31
2,201803,2018,3,ゴールド,オンライン,25-29歳,男性,0,15,2018/03/31
3,201803,2018,3,ゴールド,オンライン,25-29歳,男性,1,2,2018/03/31
4,201803,2018,3,ゴールド,オンライン,30-34歳,女性,0,17,2018/03/31


全体の入りと出を集計

In [269]:
q = """
        SELECT
        cp_date
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date
        """

df_web = sqldf(q, locals())
df_web.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,89982,6727,96709
1,2018/04/30,95522,7156,102678
2,2018/05/31,101454,8139,109593
3,2018/06/30,108303,6193,114496
4,2018/07/31,113194,6058,119252


In [270]:
#前月末を算出
q = """
        SELECT
        *
        ,LAG (eop_sum_acco, 1) OVER (ORDER BY cp_date) AS bop_sum_acco
        from 
            df_web
        group by
        cp_date
        """

df_web2 = sqldf(q, locals())
df_web2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco
0,2018/03/31,89982,6727,96709,NaN
1,2018/04/30,95522,7156,102678,96709.0
2,2018/05/31,101454,8139,109593,102678.0
3,2018/06/30,108303,6193,114496,109593.0
4,2018/07/31,113194,6058,119252,114496.0


In [271]:
df_web2['cancel_acco'] = df_web2['sum_new_acco'] - (df_web2['eop_sum_acco']-df_web2['bop_sum_acco'])
df_web2['cancel_rate'] = df_web2['cancel_acco']/df_web2['bop_sum_acco']
df_web2.head()

,cp_date,sum_exist_acco,sum_new_acco,eop_sum_acco,bop_sum_acco,cancel_acco,cancel_rate
0,2018/03/31,89982,6727,96709,NaN,NaN,NaN
1,2018/04/30,95522,7156,102678,96709.0,1187.0,0.012274
2,2018/05/31,101454,8139,109593,102678.0,1224.0,0.011921
3,2018/06/30,108303,6193,114496,109593.0,1290.0,0.011771
4,2018/07/31,113194,6058,119252,114496.0,1302.0,0.011372


In [272]:
#CSV出力
df_web2.to_csv(data_dir /f'{result}.csv', header=True ,index=None, encoding='shift=jis')

②有効口座数全体データの読込＆集計

In [273]:
#過去累積分と更新分を合算し、CSV出力
#df1 = pd.read_csv(data_dir /f"{input_all}.csv",encoding='shift-jis')
df_cust = pd.read_csv(data_dir /f"{input_cust_all}.csv", encoding='shift-jis')
#文字化けするので修正
#df1['age'] = df1['age'].str.replace('?','～')

#df2 = pd.read_csv(data_dir /f"{input_new}.csv",encoding='shift-jis')
#df = pd.concat([df1,df2])
#上書き更新するので注意
df_cust.to_csv(data_dir /f'{output2}.csv', header=True ,index=None, encoding='shift-jis')

In [274]:
df_cust = df_cust[df_cust['cp']>= 1803] #データが2019年3月以降しか現状無し
df_cust

,cp,year,month,card,channel,age,gender,no_of_account
0,1903,2019,3,ゴールド,オンライン,25-29歳,女性,28
1,1903,2019,3,ゴールド,オンライン,25-29歳,男性,29
2,1903,2019,3,ゴールド,オンライン,30-34歳,女性,37
3,1903,2019,3,ゴールド,オンライン,30-34歳,男性,73
4,1903,2019,3,ゴールド,オンライン,35-39歳,女性,55
...,...,...,...,...,...,...,...,...
2725,2005,2020,5,外商,郵送,70-74歳,男性,14757
2726,2005,2020,5,外商,郵送,75-79歳,女性,4686
2727,2005,2020,5,外商,郵送,75-79歳,男性,10108
2728,2005,2020,5,外商,郵送,80歳以上,女性,9766


In [275]:
print(df_cust.shape)
print(df_cust.index)
print(df_cust.columns)
print(df_cust.dtypes)

(2730, 8)
Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            2720, 2721, 2722, 2723, 2724, 2725, 2726, 2727, 2728, 2729],
           dtype='int64', length=2730)
Index(['cp', 'year', 'month', 'card', 'channel', 'age', 'gender',
       'no_of_account'],
      dtype='object')
cp                int64
year              int64
month             int64
card             object
channel          object
age              object
gender           object
no_of_account     int64
dtype: object


In [276]:
#日付データの変換
df_cust['cp'] = df_cust['cp'] + 200000
df_cust['cp_date'] = pd.to_datetime(df_cust['cp'], format="%Y%m") + MonthEnd(1)
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019-03-31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019-03-31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019-03-31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019-03-31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019-03-31


#日付データの変換
df_cust['cp_date'] = (df_cust['cp'] + 200000)*100 + 1
df_cust['cp_date']=pd.to_datetime(df_cust['cp_date'], format='%Y%m%d')
df_cust.head()

In [277]:
#日付から時間を削除
df_cust['cp_date'] = df_cust['cp_date'].dt.strftime('%Y/%m/%d')
df_cust.head()

,cp,year,month,card,channel,age,gender,no_of_account,cp_date
0,201903,2019,3,ゴールド,オンライン,25-29歳,女性,28,2019/03/31
1,201903,2019,3,ゴールド,オンライン,25-29歳,男性,29,2019/03/31
2,201903,2019,3,ゴールド,オンライン,30-34歳,女性,37,2019/03/31
3,201903,2019,3,ゴールド,オンライン,30-34歳,男性,73,2019/03/31
4,201903,2019,3,ゴールド,オンライン,35-39歳,女性,55,2019/03/31


チャネル別新規登録数

In [278]:
print(df['channel'].unique())

['オンライン' '店頭タブレット' '郵送']


In [279]:
#Web明細
q = """
        SELECT
        cp_date
        ,channel
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,channel
        """

df_cha = sqldf(q, locals())
df_cha.head()

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,オンライン,3436,225,3661
1,2018/03/31,店頭タブレット,52497,4522,57019
2,2018/03/31,郵送,34049,1980,36029
3,2018/04/30,オンライン,3618,220,3838
4,2018/04/30,店頭タブレット,56230,4740,60970


In [280]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,channel
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,channel
        """

df_cst_cha = sqldf(q, locals())
df_cst_cha.head()

,cp_date,channel,total_cust
0,2019/03/31,オンライン,11114
1,2019/03/31,店頭タブレット,430268
2,2019/03/31,郵送,1117369
3,2019/04/30,オンライン,11171
4,2019/04/30,店頭タブレット,430348


In [281]:
df_cha2 = pd.merge(df_cha,df_cst_cha, on =['cp_date','channel'], how = 'left')
df_cha2['regist_rate'] = df_cha2['eop_sum_acco']/df_cha2['total_cust']
df_cha2

,cp_date,channel,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,オンライン,3436,225,3661,NaN,NaN
1,2018/03/31,店頭タブレット,52497,4522,57019,NaN,NaN
2,2018/03/31,郵送,34049,1980,36029,NaN,NaN
3,2018/04/30,オンライン,3618,220,3838,NaN,NaN
4,2018/04/30,店頭タブレット,56230,4740,60970,NaN,NaN
...,...,...,...,...,...,...,...
76,2020/04/30,店頭タブレット,125988,3191,129179,438236.0,0.294770
77,2020/04/30,郵送,90243,6945,97188,1062694.0,0.091454
78,2020/05/31,オンライン,8719,704,9423,13332.0,0.706796
79,2020/05/31,店頭タブレット,127675,3694,131369,434238.0,0.302528


In [282]:
#CSV出力
df_cha2.to_csv(data_dir /f'{result_cha}.csv', header=True ,index=None, encoding='shift=jis')

カード種類別新規登録数

In [283]:
print(df['card'].unique())

['ゴールド' '一般' '外商']


In [284]:
q = """
        SELECT
        cp_date
        ,card
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,card
        """

df_card = sqldf(q, locals())
df_card.head()

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,ゴールド,5016.0,352.0,5368
1,2018/03/31,一般,84966.0,6375.0,91341
2,2018/04/30,ゴールド,5292.0,383.0,5675
3,2018/04/30,一般,90230.0,6773.0,97003
4,2018/05/31,ゴールド,5605.0,466.0,6071


In [285]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,card
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,card
        """

df_cst_card = sqldf(q, locals())
df_cst_card.head()

,cp_date,card,total_cust
0,2019/03/31,ゴールド,42171
1,2019/03/31,一般,1361663
2,2019/03/31,外商,154917
3,2019/04/30,ゴールド,42317
4,2019/04/30,一般,1356056


In [286]:
df_card2 = pd.merge(df_card,df_cst_card, on =['cp_date','card'], how = 'left')
df_card2['regist_rate'] = df_card2['eop_sum_acco']/df_card2['total_cust']
df_card2

,cp_date,card,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,ゴールド,5016.0,352.0,5368,NaN,NaN
1,2018/03/31,一般,84966.0,6375.0,91341,NaN,NaN
2,2018/04/30,ゴールド,5292.0,383.0,5675,NaN,NaN
3,2018/04/30,一般,90230.0,6773.0,97003,NaN,NaN
4,2018/05/31,ゴールド,5605.0,466.0,6071,NaN,NaN
...,...,...,...,...,...,...,...
70,2020/04/30,一般,200052.0,8495.0,208547,1306799.0,0.159586
71,2020/04/30,外商,13142.0,1381.0,14523,164342.0,0.088371
72,2020/05/31,ゴールド,12028.0,677.0,12705,42515.0,0.298836
73,2020/05/31,一般,206554.0,12193.0,218747,1298675.0,0.168439


In [287]:
#CSV出力
df_card2.to_csv(data_dir /f'{result_card}.csv', header=True ,index=None, encoding='shift=jis')

年令別

In [288]:
print(df['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '80歳以上' '75-79歳' '19歳以下（20歳未満）' '20-24歳']


In [289]:
q = """
        SELECT
        cp_date
        ,age
        ,sum(case when new_account= 0 then no_of_account end) as sum_exist_acco
        ,sum(case when new_account= 1 then no_of_account end) as sum_new_acco
        ,sum(no_of_account) as eop_sum_acco
        from 
            df
        group by
        cp_date,age
        """

df_age = sqldf(q, locals())
df_age.head()

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco
0,2018/03/31,19歳以下（20歳未満）,163,32,195
1,2018/03/31,20-24歳,7577,881,8458
2,2018/03/31,25-29歳,12587,1001,13588
3,2018/03/31,30-34歳,11282,793,12075
4,2018/03/31,35-39歳,9719,648,10367


In [290]:
print(df_cust['age'].unique())

['25-29歳' '30-34歳' '35-39歳' '40-44歳' '45-49歳' '50-54歳' '55-59歳' '60-64歳'
 '65-69歳' '70-74歳' '75-79歳' '80歳以上' '19歳以下（20歳未満）' '20-24歳']


In [291]:
#口座数全体を同じレイヤーで集計する
q = """
        SELECT
        cp_date
        ,age
        ,sum(no_of_account) as total_cust
        from 
            df_cust
        group by
        cp_date,age
        """

df_cst_age = sqldf(q, locals())
df_cst_age.head()

,cp_date,age,total_cust
0,2019/03/31,19歳以下（20歳未満）,478
1,2019/03/31,20-24歳,23281
2,2019/03/31,25-29歳,52262
3,2019/03/31,30-34歳,68756
4,2019/03/31,35-39歳,84421


In [292]:
df_age2 = pd.merge(df_age,df_cst_age, on =['cp_date','age'], how = 'left')
df_age2['regist_rate'] = df_age2['eop_sum_acco']/df_age2['total_cust']
df_age2

,cp_date,age,sum_exist_acco,sum_new_acco,eop_sum_acco,total_cust,regist_rate
0,2018/03/31,19歳以下（20歳未満）,163,32,195,NaN,NaN
1,2018/03/31,20-24歳,7577,881,8458,NaN,NaN
2,2018/03/31,25-29歳,12587,1001,13588,NaN,NaN
3,2018/03/31,30-34歳,11282,793,12075,NaN,NaN
4,2018/03/31,35-39歳,9719,648,10367,NaN,NaN
...,...,...,...,...,...,...,...
373,2020/05/31,60-64歳,18259,1956,20215,159325.0,0.126879
374,2020/05/31,65-69歳,10846,1418,12264,161136.0,0.076110
375,2020/05/31,70-74歳,6267,954,7221,169749.0,0.042539
376,2020/05/31,75-79歳,2156,258,2414,117292.0,0.020581


In [293]:
#CSV出力
df_age2.to_csv(data_dir /f'{result_age}.csv', header=True ,index=None, encoding='utf-8-sig')